### Feature Selection 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import DBSCAN, KMeans
import hdbscan
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA

In [3]:
df= pd.read_csv('./Data/res_time.csv')

In [4]:
df.head(5)

,F_SESSION,0,1,2,3,4,5,6,7,8,...,18644,18645,18646,18647,18648,18649,18650,18651,18652,18653
0,3130311,300100285,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3130519,300100285,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3130827,300100285,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3131023,300100285,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3131064,300100285,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,300100285.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1837 entries, 0 to 1836
Columns: 18655 entries, F_SESSION to 18653
dtypes: float64(18653), int64(2)
memory usage: 261.5 MB


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.utils import to_time_series_dataset

# 1. Préparation des données
# tslearn attend un format 3D (n_vols, n_secondes, n_variables)
# Si ton dataframe 'df' a chaque seconde en colonne :
X = df.values 

# Conversion au format tslearn (gère les longueurs différentes si besoin)
X_train = to_time_series_dataset(X)

# 2. Normalisation (Z-score scaling par série temporelle)
# Indispensable pour que la forme de la courbe compte plus que les valeurs absolues
X_train = TimeSeriesScalerMeanVariance().fit_transform(X_train)

# 3. Création et entraînement du modèle
# metric="dtw" ou "softdtw" (plus rapide et différentiable)
km = TimeSeriesKMeans(n_clusters=3, 
                      metric="dtw", 
                      max_iter=10,
                      random_state=42,
                      n_jobs=-1) # Utilise tous les processeurs de ton PC

labels = km.fit_predict(X_train)

# 4. Visualisation des résultats (Centroïdes)
plt.figure(figsize=(12, 8))
for cluster in range(3):
    plt.subplot(3, 1, cluster + 1)
    # On affiche les vols du cluster en gris
    for x in X_train[labels == cluster]:
        plt.plot(x.ravel(), "k-", alpha=.2)
    # On affiche le "vol moyen" (centroïde) du cluster en rouge
    plt.plot(km.cluster_centers_[cluster].ravel(), "r-", linewidth=3)
    plt.title(f"Cluster {cluster + 1}")

plt.tight_layout()
plt.show()

/home/mohamed/anaconda3/envs/myenv/lib/python3.11/site-packages/tslearn/bases/bases.py:16: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)
